<a href="https://colab.research.google.com/github/basavarajmullur/Spring-Boot-JdbcTemplate/blob/master/notebooks/medgemma-interfencing-api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

!pip install -q transformers accelerate bitsandbytes
!pip install -q fastapi uvicorn pyngrok pillow


## Setup

To complete this tutorial, you'll need to have a runtime with [sufficient resources](https://ai.google.dev/gemma/docs/core#sizes) to run the MedGemma model.

You can try out MedGemma 4B for free in Google Colab using a T4 GPU:

1. In the upper-right of the Colab window, select **▾ (Additional connection options)**.
2. Select **Change runtime type**.
3. Under **Hardware accelerator**, select **T4 GPU**.

**Note**: To run the demo with MedGemma 27B in Google Colab, you will need a runtime with an A100 GPU.

### Get access to MedGemma

Before you get started, make sure that you have access to MedGemma models on Hugging Face:

1. If you don't already have a Hugging Face account, you can create one for free by clicking [here](https://huggingface.co/join).
2. Head over to the [MedGemma model page](https://huggingface.co/google/medgemma-1.5-4b-it) and accept the usage conditions.

### Step 1: Authenticate with Hugging Face


In [1]:
from huggingface_hub import login
login()

### Step 2: Install dependencies

In [2]:
!pip install -q \
  fastapi \
  uvicorn \
  transformers \
  accelerate \
  bitsandbytes \
  pillow==10.4.0 \
  torch torchvision \





## Step 3: Load MedGemma

In [ ]:
import torch
from transformers import AutoProcessor, AutoModelForCausalLM

MODEL_ID = "google/medgemma-4b-it"

processor = AutoProcessor.from_pretrained(MODEL_ID)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.float16,
    device_map="auto"
)

model.eval()
print("✅ MedGemma loaded")


The image processor of type `Gemma3ImageProcessor` is now loaded as a fast processor by default, even if the model checkpoint was saved with a slow processor. This is a breaking change and may produce slightly different outputs. To continue using the slow processor, instantiate this class with `use_fast=False`. 
`torch_dtype` is deprecated! Use `dtype` instead!


Loading weights:   0%|          | 0/883 [00:00<?, ?it/s]

## Step 4: Install cloudflared

In [ ]:
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared-linux-amd64

## Step 5: MEDICAL SYSTEM PROMPT

In [ ]:
from fastapi import FastAPI, UploadFile, Form
from PIL import Image
import io
import json
from pydantic import BaseModel
from typing import Optional
import base64
from fastapi import HTTPException
from PIL import Image
import io
import base64
import traceback
import base64
import io
from PIL import Image

def base64_to_pil(image_base64: str) -> Image.Image:
    """
    Convert a base64 string (raw or data URL) into a PIL Image.

    Supports:
    - Raw base64 (no prefix)
    - data:image/png;base64,...
    - data:image/jpeg;base64,...
    """

    if not image_base64:
        raise ValueError("Empty base64 image string")

    # Strip data URL prefix if present
    if image_base64.startswith("data:"):
        image_base64 = image_base64.split(",", 1)[1]

    try:
        image_bytes = base64.b64decode(image_base64, validate=True)
    except Exception as e:
        raise ValueError("Invalid base64 image data") from e

    try:
        image = Image.open(io.BytesIO(image_bytes))
        image = image.convert("RGB")
    except Exception as e:
        raise ValueError("Decoded bytes are not a valid image") from e

    return image

class AnalyzeRequest(BaseModel):
    prompt: str
    image_base64: str
    max_tokens: int = 512

def decode_base64_image(data_url: str) -> bytes:
    if "," in data_url:
        data_url = data_url.split(",", 1)[1]
    return base64.b64decode(data_url)

app = FastAPI(title="ClinIQ – MedGemma API")

SYSTEM_PROMPT = """
You are a clinical decision support assistant.

Rules:
- Do NOT provide diagnoses
- Use observational language only
- Explicitly state uncertainty
- Phrase findings for clinicians
- Avoid prescriptive advice

Respond ONLY with valid JSON.

JSON schema:
{
  "observations": [],
  "possible_interpretations": [],
  "uncertainty_notes": "",
  "recommend_next_steps": []
}
"""
@app.post("/debug")

def run_medgemma(image, prompt, max_tokens):
    messages = [
        {
            "role": "user",
            "content": [
                {"type": "image"},
                {
                    "type": "text",
                    "text": (
                        f"{prompt}\n\n"
                        "Respond now with the JSON output only."
                    )
                }
            ]
        }
    ]

    formatted_prompt = processor.apply_chat_template(
        messages,
        add_generation_prompt=True
    )

    inputs = processor(
        text=formatted_prompt,
        images=[image],
        return_tensors="pt"
    )

    input_ids = inputs["input_ids"]
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_tokens,
            do_sample=True,          # 🔑 IMPORTANT
            temperature=0.3,         # 🔑 Nudge generation
            top_p=0.9,
            eos_token_id=processor.tokenizer.eos_token_id
        )

    generated_tokens = outputs[0][input_ids.shape[-1]:]

    # 🔍 HARD GUARD
    if generated_tokens.numel() == 0:
        return "⚠️ Model returned no output. Try simplifying the prompt."

    return processor.decode(
        generated_tokens,
        skip_special_tokens=True
    ).strip()


@app.post("/analyze")
def analyze(req: AnalyzeRequest):
    try:
        image_bytes = decode_base64_image(req.image_base64)
        image = Image.open(io.BytesIO(image_bytes)).convert("RGB")

        result = run_medgemma(
            image=image,
            prompt=req.prompt,
            max_tokens=req.max_tokens
        )

        return {"response": result}

    except Exception as e:
        print("❌ ANALYZE FAILED")
        traceback.print_exc()
        raise HTTPException(status_code=422, detail=str(e))




## Step 6: Run FastAPI server



In [ ]:
import logging
import uvicorn
from threading import Thread

# -----------------------
# Logging setup
# -----------------------
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(name)s | %(message)s"
)

logger = logging.getLogger("cliniq")

def start_api():
    logger.info("Starting FastAPI server on 127.0.0.1:8000")

    uvicorn.run(
        app,
        host="127.0.0.1",
        port=8000,
        log_level="info",
        access_log=True
    )

    logger.info("Uvicorn process exited")

Thread(target=start_api).start()


## Step 7 Expose via Cloudflare Tunnel

In [7]:
import subprocess
import re

process = subprocess.Popen(
    [
        "./cloudflared-linux-amd64",
        "tunnel",
        "--no-autoupdate",
        "--protocol", "http2",        # ❌ no QUIC
        "--url", "http://127.0.0.1:8000"
    ],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True,
)

for line in process.stdout:
    print(line, end="")
    if "trycloudflare.com" in line:
        print("\n🌍 COPY THIS URL ↑↑↑\n")


KeyboardInterrupt: 